In [41]:
# Install datacommons_pandas
# !pip install datacommons_pandas --upgrade --quiet
# Import Data Commons
import datacommons_pandas as dc

# Import other required libraries
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
from datetime import datetime
import json
import io
import requests

In [19]:
def add_name_col(df):
  # Add a new column called name, where each value is the name for the place dcid in the index.
  df['name'] = df.index.map(dc.get_property_values(df.index, 'name'))
  
  # Keep just the first name, instead of a list of all names.
  df['name'] = df['name'].str[0]

In [2]:
usa = 'country/USA'

# Get lists of states, counties, and cities within the United States, respectively.
states = dc.get_places_in([usa], 'State')[usa]
counties = dc.get_places_in([usa], 'County')[usa]
cities = dc.get_places_in([usa], 'City')[usa]

In [3]:
states[:10]

['geoId/01',
 'geoId/02',
 'geoId/04',
 'geoId/05',
 'geoId/06',
 'geoId/08',
 'geoId/09',
 'geoId/10',
 'geoId/11',
 'geoId/12']

In [20]:
df_unemp_state = dc.build_time_series_dataframe(states, 'UnemploymentRate_Person', desc_col=True)
df_unemp_state.head()

,2024-06,2024-05,2024-04,2024-03,2024-02,2024-01,2023-12,2023-11,2023-10,2023-09,...,1976-10,1976-09,1976-08,1976-07,1976-06,1976-05,1976-04,1976-03,1976-02,1976-01
place,,,,,,,,,,,,,,,,,,,,,
geoId/01,2.9,2.4,2.6,3.0,3.4,3.4,2.7,2.5,2.6,2.6,...,6.6,6.7,7.1,7.2,7.4,5.2,5.7,6.3,7.7,7.5
geoId/02,4.9,4.2,4.5,4.8,5.5,5.2,4.6,4.5,4.2,4.0,...,6.9,6.6,5.4,5.8,7.3,6.5,7.5,8.7,9.7,9.3
geoId/04,3.9,3.2,2.6,2.9,3.6,3.8,3.8,3.8,4.2,4.2,...,9.7,10.0,10.4,10.3,10.5,8.4,9.2,9.8,10.6,11.5
geoId/05,3.7,3.3,3.1,3.3,3.5,3.9,3.4,3.2,3.1,3.5,...,5.5,6.1,6.8,7.2,7.3,6.2,6.7,7.5,8.6,8.8
geoId/06,5.3,4.6,4.9,5.4,5.6,5.7,5.1,4.9,5.0,4.9,...,8.8,8.8,9.1,9.5,9.0,7.9,8.8,9.4,10.1,10.4


In [21]:
add_name_col(df_unemp_state)

# set name column as first column
cols = df_unemp_state.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_unemp_state = df_unemp_state[cols]

df_unemp_state.head()

,name,2024-06,2024-05,2024-04,2024-03,2024-02,2024-01,2023-12,2023-11,2023-10,...,1976-10,1976-09,1976-08,1976-07,1976-06,1976-05,1976-04,1976-03,1976-02,1976-01
place,,,,,,,,,,,,,,,,,,,,,
geoId/01,Alabama,2.9,2.4,2.6,3.0,3.4,3.4,2.7,2.5,2.6,...,6.6,6.7,7.1,7.2,7.4,5.2,5.7,6.3,7.7,7.5
geoId/02,Alaska,4.9,4.2,4.5,4.8,5.5,5.2,4.6,4.5,4.2,...,6.9,6.6,5.4,5.8,7.3,6.5,7.5,8.7,9.7,9.3
geoId/04,Arizona,3.9,3.2,2.6,2.9,3.6,3.8,3.8,3.8,4.2,...,9.7,10.0,10.4,10.3,10.5,8.4,9.2,9.8,10.6,11.5
geoId/05,Arkansas,3.7,3.3,3.1,3.3,3.5,3.9,3.4,3.2,3.1,...,5.5,6.1,6.8,7.2,7.3,6.2,6.7,7.5,8.6,8.8
geoId/06,California,5.3,4.6,4.9,5.4,5.6,5.7,5.1,4.9,5.0,...,8.8,8.8,9.1,9.5,9.0,7.9,8.8,9.4,10.1,10.4


In [42]:
# filter for months from 2017-01 to {current year}-{current month - 1}
current_year = datetime.now().year
current_month = datetime.now().month

# Calculate the previous month
if current_month == 1:
    previous_month_year = current_year - 1
    previous_month = 12
else:
    previous_month_year = current_year
    previous_month = current_month - 1


date_range = pd.date_range(start='2017-01', end=f'{previous_month_year}-{previous_month}', freq='MS').strftime('%Y-%m').tolist()
filtered_cols = ['name'] + [col for col in df_unemp_state.columns if col in date_range]

print(date_range)
print(filtered_cols)

['2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12', '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12', '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06']
['name', 

In [36]:
df_filtered = df_unemp_state[filtered_cols]

In [37]:
df_filtered.head()

,name,2024-06,2024-05,2024-04,2024-03,2024-02,2024-01,2023-12,2023-11,2023-10,...,2017-10,2017-09,2017-08,2017-07,2017-06,2017-05,2017-04,2017-03,2017-02,2017-01
place,,,,,,,,,,,,,,,,,,,,,
geoId/01,Alabama,2.9,2.4,2.6,3.0,3.4,3.4,2.7,2.5,2.6,...,3.8,3.9,4.5,4.7,4.9,4.1,4.3,5.0,5.6,6.0
geoId/02,Alaska,4.9,4.2,4.5,4.8,5.5,5.2,4.6,4.5,4.2,...,5.9,5.9,5.9,6.1,6.5,6.3,6.6,7.0,7.4,7.5
geoId/04,Arizona,3.9,3.2,2.6,2.9,3.6,3.8,3.8,3.8,4.2,...,4.7,4.9,5.3,5.3,5.2,4.8,4.7,4.9,5.1,5.4
geoId/05,Arkansas,3.7,3.3,3.1,3.3,3.5,3.9,3.4,3.2,3.1,...,3.2,3.5,3.9,4.1,3.9,3.6,3.2,3.6,4.4,4.4
geoId/06,California,5.3,4.6,4.9,5.4,5.6,5.7,5.1,4.9,5.0,...,4.3,4.5,5.1,5.2,4.9,4.4,4.7,5.2,5.5,5.7


In [38]:
# pivot the table so that each row represents the unemployment rate for a state for a given month

# id_vars=['name']: Specifies the column(s) to keep as identifier variables.
# var_name='month_of_measurement': Specifies the name for the variable column.
# value_name='unemployment_rate': Specifies the name for the value column.
pd.melt(df_filtered, id_vars=['name'], var_name='month_of_measurement', value_name='unemployment_rate')

,name,month_of_measurement,unemployment_rate
0,Alabama,2024-06,2.9
1,Alaska,2024-06,4.9
2,Arizona,2024-06,3.9
3,Arkansas,2024-06,3.7
4,California,2024-06,5.3
...,...,...,...
4675,Washington,2017-01,5.5
4676,West Virginia,2017-01,6.4
4677,Wisconsin,2017-01,3.9
4678,Wyoming,2017-01,5.5
